# Merging DataFrames

In [2]:
import pandas as pd

In [3]:
df = pd.DataFrame([{'Name': 'Cris', 'Item Purchased': 'Sponge', 'Cost': 22.50},
                 {'Name': 'Kevyn', 'Item Purchased': 'Kitty Litter', 'Cost': 2.50},
                 {'Name': 'Filip', 'Item Purchased': 'Spoon', 'Cost': 5.00}],
                 index=['Store 1', 'Store 1', 'Store 2'])

df

,Cost,Item Purchased,Name
Store 1,22.5,Sponge,Cris
Store 1,2.5,Kitty Litter,Kevyn
Store 2,5.0,Spoon,Filip


Para adicionarmos uma coluna com novos dados em um DataFrame, basta usarmos o operador []. Desde que o tamanho do array de dados recebido seja o mesmo do número de linhas da tabela, isso funcionará.

In [5]:
df['Date'] = ['December 1', 'January 1', 'mid-May']
df

,Cost,Item Purchased,Name,Date
Store 1,22.5,Sponge,Cris,December 1
Store 1,2.5,Kitty Litter,Kevyn,January 1
Store 2,5.0,Spoon,Filip,mid-May


Se quisermos adicionar apenas uma nova coluna, como por exemplo uma flag, também podemos fazer isso usando o operador [], desde que o valor atribuído seja um escalar (um número, uma string, um booleano ou None).

In [6]:
df['Flag'] = True
df

,Cost,Item Purchased,Name,Date,Flag
Store 1,22.5,Sponge,Cris,December 1,True
Store 1,2.5,Kitty Litter,Kevyn,January 1,True
Store 2,5.0,Spoon,Filip,mid-May,True


O problema é quando queremos criar uma coluna, mas queremos adicionar apenas alguns itens na mesma. Como falamos antes, para esse procedimento funcionar, devemos dar como parâmetro ao *pandas* uma lista que seja longa o bastante para suprir todas as linhas do DataFrame com seus valores. Por isso devemos colocar nesse array por nós mesmo **None** nas posições das linhas que não desejamos incluir valores.

Entretanto, se cada linha do DataFrame possui um valor único, podemos usá-los para especificar as linhas que desejamos colocar valores ao adicionar uma coluna nova. O bom dessa abordagem é que podemos simplesmente ignorar as linhas que não desejarmos incluir valores, e o *pandas* as preencherá com **valores ausentes**.

In [8]:
adf = df.reset_index()
adf ['Date'] = pd.Series({0: 'December 1', 2: 'mid-May'})
adf

,index,Cost,Item Purchased,Name,Date,Flag
0,Store 1,22.5,Sponge,Cris,December 1,True
1,Store 1,2.5,Kitty Litter,Kevyn,NaN,True
2,Store 2,5.0,Spoon,Filip,mid-May,True


Abaixo temos duas tabelas, uma de funcionários e outra de estudantes. Percebemos que um dos estudantes, o James, está alocado em ambas as tabelas, indicando que ele é tanto funcionário quanto estudante. Além disso, percebemos que a coluna *Name* foi definida como índice, sendo também ela que usaremos para operações de junção (join). 

In [22]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR'},
                       {'Name': 'Sally', 'Role': 'Course Liaison'},
                       {'Name': 'James', 'Role': 'Grader'}])
staff_df = staff_df.set_index('Name')
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business'},
                          {'Name': 'Mike', 'School': 'Law'},
                          {'Name': 'Sally', 'School': 'Engineering'}])
student_df = student_df.set_index('Name')

print(staff_df)
print()
print(student_df)

                 Role
Name                 
Kelly  Director of HR
Sally  Course Liaison
James          Grader

            School
Name              
James     Business
Mike           Law
Sally  Engineering


Para juntarmos as duas tabelas acima, usamos a função **`.merge`** na qual inserimos como parâmetro, respectivamente, as duas tabelas a serem unidas, o método de junção (que no exemplo será o **outer**, ou união), e a definição de que o índice das tabelas da direita e da esquerda devem ou não ser o mesmo índice da união. 

In [23]:
pd.merge(staff_df, student_df, how='outer', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Kelly,Director of HR,NaN
Mike,NaN,Law
Sally,Course Liaison,Engineering


Na união das duas tabelas atribuímos ao parâmetro how **outer**. Agora queremos apenas aqueles que são tanto funcionários quanto estudantes, isto é, a interseção de staff_df e student_df. Para isso, atribuímos ao parâmetro how **inner**.

In [24]:
pd.merge(staff_df, student_df, how='inner', left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Sally,Course Liaison,Engineering


Agora queremos pegar as informações de todos os funcionários, independentemente se eles são estudantes ou não, porém se forem estudantes queremos saber em qual curso estão matriculados. Para isso, fazemos um left-join atribuindo ao parâmetro how **left**.

In [25]:
pd.merge(staff_df, student_df, how="left", left_index=True, right_index=True)

,Role,School
Name,,
Kelly,Director of HR,NaN
Sally,Course Liaison,Engineering
James,Grader,Business


E o que falta agora é pegar informações de todos os estudantes e seus cargos caso também sejam funcionários. Para isso, fazemos um right-join atribuindo ao parâmetro how **right**.

In [26]:
pd.merge(staff_df, student_df, how="right", left_index=True, right_index=True)

,Role,School
Name,,
James,Grader,Business
Mike,NaN,Law
Sally,Course Liaison,Engineering


Nós não precisamos usar os índices para realizar a junção de tabelas. Podemos usar as colunas também através dos atributos **`left_on`** e **`right_on`**.

In [27]:
staff_df = staff_df.reset_index()
student_df = student_df.reset_index()
pd.merge(staff_df, student_df, how='left', left_on='Name', right_on='Name')

,Name,Role,School
0,Kelly,Director of HR,NaN
1,Sally,Course Liaison,Engineering
2,James,Grader,Business


O que será que acontece quando temos conflitos entre DataFrames?

In [29]:
staff_df = pd.DataFrame([{'Name': 'Kelly', 'Role': 'Director of HR', 'Location': 'State Street'},
                        {'Name': 'Sally', 'Role': 'Course Liaison', 'Location': 'Washington Avenue'},
                        {'Name': 'James', 'Role': 'Grader', 'Location': 'Washington Avenue'}])
student_df = pd.DataFrame([{'Name': 'James', 'School': 'Business', 'Location': '1024 Billiard Avenue'},
                          {'Name': 'Mike', 'School': 'Law', 'Location': 'Fraternity House #22'},
                          {'Name': 'Sally', 'School': 'Engineering', 'Location': '512 Wilson Crescent'}])
pd.merge(staff_df, student_df, how='left', left_on='Name', right_on='Name')

,Location_x,Name,Role,Location_y,School
0,State Street,Kelly,Director of HR,NaN,NaN
1,Washington Avenue,Sally,Course Liaison,512 Wilson Crescent,Engineering
2,Washington Avenue,James,Grader,1024 Billiard Avenue,Business


Como podemos ver, foram anexados nos nomes das colunas *Location* **_x** e **_y** como uma maneira de diferenciar as duas. Enquanto Location de *staff_df* se refere ao local de trabalho, Location de *student_df* se refere ao endereço de casa. Devemos sempre lembrar que **_x** é sempre anexado a informações da tabela da esquerda e **_y** é anexado a informações da tabela da direita. Apesar disso, podemos inserir nomes diferentes para **_x** e **_y** através de parâmetros caso desejamos.

Agpra vejamos o caso no qual precisamos fazer um join com múltiplas colunas. Nos DataFrames abaixo a coluna *First Name* podem se sobrepor, mas a coluna *Last Name* não. Nesse caso, atribui-se aos parâmetros **`left_on`** e **`right_on`** uma lista de colunas para serem as chaves da junção (join keys)

In [5]:
staff_df = pd.DataFrame([{'First Name': 'Kelly', 'Last Name': 'Desjardins', 'Role': 'Director of HR'},
                         {'First Name': 'Sally', 'Last Name': 'Brooks', 'Role': 'Course Liaison'},
                         {'First Name': 'James', 'Last Name': 'Wilde', 'Role': 'Grader'}])

student_df = pd.DataFrame([{'First Name': 'James', 'Last Name': 'Hammond', 'School': 'Business'},
                           {'First Name': 'Mike', 'Last Name': 'Smith', 'School': 'Law'},
                           {'First Name': 'Sally', 'Last Name': 'Brooks', 'School': 'Engineering'}])

print(staff_df)
print()
print(student_df)
print()
print(pd.merge(staff_df, student_df, how='inner', left_on=['First Name', 'Last Name'], right_on=['First Name', 'Last Name']))

  First Name   Last Name            Role
0      Kelly  Desjardins  Director of HR
1      Sally      Brooks  Course Liaison
2      James       Wilde          Grader

  First Name Last Name       School
0      James   Hammond     Business
1       Mike     Smith          Law
2      Sally    Brooks  Engineering

  First Name Last Name            Role       School
0      Sally    Brooks  Course Liaison  Engineering
